# Pembangunan Model Akustik Langsung - Sistem Pengenal Emosi Cakapan

## Library Preparation

In [1]:
!pip install arff
!pip install imbalanced-learn

  Created wheel for arff: filename=arff-0.9-cp37-none-any.whl size=4970 sha256=8e20f0b69cf274a26201d30a90da2384a53fba748fc57c4a172ab420c853c6cd
  Stored in directory: /root/.cache/pip/wheels/04/d0/70/2c73afedd3ac25c6085b528742c69b9587cbdfa67e5194583b
Successfully built arff


In [2]:
import arff
import joblib
from io import StringIO
from google.colab import drive
from google.colab import files

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras import layers
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
drive.mount('/content/gdrive')
drive_dir = "/content/gdrive/MyDrive/Teknik Informatika/Semester 7/TUGAS AKHIR/TA 2"

Mounted at /content/gdrive


## Feature Extraction

### Load Extracted Audio Features

In [4]:
features_path = drive_dir + "/Eksperimen/Data"

In [5]:
feature_IS09 = arff.load(features_path + '/IS09.arff')
feature_IS10 = arff.load(features_path + '/IS10.arff')
feature_IS13 = arff.load(features_path + '/IS13.arff')
feature_eGeMAPS = arff.load(features_path + '/eGeMAPS.arff')
feature_emobase = arff.load(features_path + '/emoLarge.arff')

In [6]:
def read_arff_data(data):
  row = next(data)
  row = next(data)
  list_data = []
  i = 0
  while row:
    if (i % 1000 == 0):
      print('Reading...', i, 'data already done.')
    row_data = []
    for j in range(len(row)):
      if (j != 0 and row[j] != ''):
        row_data.append(row[j])
    list_data.append(row_data)
    try :
      row = next(data)
      i = i + 1
    except :
      break
  return np.array(list_data)

In [7]:
feature_IS09 = read_arff_data(feature_IS09)
feature_IS10 = read_arff_data(feature_IS10)
feature_IS13 = read_arff_data(feature_IS13)
feature_eGeMAPS = read_arff_data(feature_eGeMAPS)
feature_emobase = read_arff_data(feature_emobase)

Reading... 0 data already done.
Reading... 1000 data already done.
Reading... 2000 data already done.
Reading... 3000 data already done.
Reading... 4000 data already done.
Reading... 5000 data already done.
Reading... 6000 data already done.
Reading... 7000 data already done.
Reading... 8000 data already done.
Reading... 9000 data already done.
Reading... 10000 data already done.
Reading... 0 data already done.
Reading... 1000 data already done.
Reading... 2000 data already done.
Reading... 3000 data already done.
Reading... 4000 data already done.
Reading... 5000 data already done.
Reading... 6000 data already done.
Reading... 7000 data already done.
Reading... 8000 data already done.
Reading... 9000 data already done.
Reading... 10000 data already done.
Reading... 0 data already done.
Reading... 1000 data already done.
Reading... 2000 data already done.
Reading... 3000 data already done.
Reading... 4000 data already done.
Reading... 5000 data already done.
Reading... 6000 data alread

In [8]:
print('Features IS09', feature_IS09.shape)
print('Features IS10', feature_IS10.shape)
print('Features IS13', feature_IS13.shape)
print('Features eGeMAPS', feature_eGeMAPS.shape)
print('Features emobase', feature_emobase.shape)

Features IS09 (10822, 384)
Features IS10 (10822, 1582)
Features IS13 (10822, 6373)
Features eGeMAPS (10822, 88)
Features emobase (10822, 6552)


In [9]:
processed_data = joblib.load(drive_dir + '/Eksperimen/Data/basic.pkl')

## Data Preparation

### Construct Sequential Data 

In [10]:
def construct_sequential(id, data):
  sequential_datas = []
  new_data= []
  for i in range(len(id)):
    if id[i].endswith('001'):
      sequential_datas.append(new_data)
      new_data = []
    new_data.append(data[i])
  sequential_datas.append(new_data)

  return np.array(sequential_datas, dtype=object)[1:]

In [11]:
def construct_abstract(id, abstraksi):
  new_abstraksi = []
  for i in range(len(id)):
    if id[i].endswith('001'):
      new_abstraksi.append(abstraksi[i])

  return np.array(new_abstraksi, dtype=object)

In [12]:
id = processed_data['ID Ucapan']
sequential_features_IS09 , sequential_features_IS10, sequential_features_IS13, sequential_features_eGeMAPS, sequential_features_emobase = construct_sequential(id, feature_IS09), construct_sequential(id, feature_IS10), construct_sequential(id, feature_IS13), construct_sequential(id, feature_eGeMAPS), construct_sequential(id, feature_emobase) 
print(sequential_features_IS09.shape, sequential_features_IS10.shape, sequential_features_IS13.shape, sequential_features_eGeMAPS.shape, sequential_features_emobase.shape)

(2003,) (2003,) (2003,) (2003,) (2003,)


In [13]:
abstraksi = construct_abstract(id, processed_data['Abstraksi Emosi'])
print(abstraksi.shape)

(2003,)


### Label One Hot Encoding

In [14]:
encoder_cakapan = OneHotEncoder(sparse=False)
abstraksi = abstraksi.reshape(-1, 1)
abstraksi = encoder_cakapan.fit_transform(abstraksi)
abstraksi.shape

(2003, 6)

### Split Train and Test Data

In [15]:
X_IS09 = sequential_features_IS09
X_IS10 = sequential_features_IS10
X_IS13 = sequential_features_IS13
X_eGeMAPS = sequential_features_eGeMAPS
X_emobase = sequential_features_emobase
y = abstraksi

In [16]:
X_IS09 = tf.keras.preprocessing.sequence.pad_sequences(X_IS09, padding="post", dtype='float32')
X_IS10 = tf.keras.preprocessing.sequence.pad_sequences(X_IS10, padding="post", dtype='float32')
X_IS13 = tf.keras.preprocessing.sequence.pad_sequences(X_IS13, padding="post", dtype='float32')
X_eGeMAPS = tf.keras.preprocessing.sequence.pad_sequences(X_eGeMAPS, padding="post", dtype='float32')
X_emobase = tf.keras.preprocessing.sequence.pad_sequences(X_emobase, padding="post", dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(y, padding="post", dtype='float32')
print("IS09", X_IS09.shape, "IS10", X_IS10.shape, "IS13", X_IS13.shape, "eGeMAPS", X_eGeMAPS.shape, "emo", X_emobase.shape)
print("Abstract", y.shape)

IS09 (2003, 15, 384) IS10 (2003, 15, 1582) IS13 (2003, 15, 6373) eGeMAPS (2003, 15, 88) emo (2003, 15, 6552)
Abstract (2003, 6)


In [17]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X_IS09, y):
  train_index, test_index = train_index, test_index
print("Train data", train_index.shape)
print("Test data", test_index.shape)

Train data (1702,)
Test data (301,)


## Modelling Preparation

### Define Model

In [18]:
def get_gru_model(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.GRU(32))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [19]:
def get_model_1(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(32))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [20]:
def get_model_2(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(128))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [21]:
def get_model_3(shape, label_shape):
  model = Sequential()
  model.add(layers.Masking(mask_value=0, input_shape=shape))
  model.add(layers.LSTM(128))
  model.add(layers.Dropout(0.2))
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dense(label_shape, activation='softmax'))
  return model

In [22]:
def create_model(shape, label_shape, mode):
  model = None
  if mode == 1:
    model = get_model_1(shape, label_shape)
  elif mode == 2 :
    model = get_model_2(shape, label_shape)
  elif mode == 3 :
    model = get_model_3(shape, label_shape)
  elif mode == "lstm":
    model = get_lstm_model(shape, label_shape)
  elif mode == "gru":
    model = get_gru_model(shape, label_shape)
  return model

### Define Training Parameter

In [23]:
batch_size = 10
epochs = 10
validation_split = 0.2
optimizer = 'rmsprop'
loss = 'categorical_crossentropy'
metrics = ['accuracy']
callbacks = [EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=3, restore_best_weights=True)]

### Define Schema Experiment

In [24]:
def train_5fold_cv(X, y, shape, label_shape, mode):

  loss_score = []
  accuracy_score = []  

  cv = KFold(n_splits=5, random_state=42, shuffle=True)

  for train_index, val_index in cv.split(X):
      
      X_train, X_val, y_train, y_val = X[train_index], X[val_index], y[train_index], y[val_index]
      
      model = create_model(shape, label_shape, mode)
      model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
      model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=epochs,
              batch_size=batch_size,
              callbacks=callbacks)
      
      l, a = model.evaluate(X_val, y_val)
      loss_score.append(l)
      accuracy_score.append(a)
  
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)
  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  return loss_score, accuracy_score

In [25]:
def train_stratified_random_sampling(X_train, y_train, X_test, y_test, shape, label_shape, mode, encoder):

  model = create_model(shape, label_shape, mode)
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
  model.fit(X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks)
      
  loss_score, accuracy_score = model.evaluate(X_test, y_test)
  loss_score = np.round(np.mean(loss_score), 4)
  accuracy_score = np.round(np.mean(accuracy_score), 4)

  pred = model.predict(X_test)
  y_pred = encoder.inverse_transform(pred)
  y_true = encoder.inverse_transform(y_test)

  print("Loss Score : ", loss_score, "Accuracy Score : ", accuracy_score)
  print(classification_report(y_true, y_pred, digits=4))
  print(confusion_matrix(y_true, y_pred))
  
  return model, accuracy_score

## Tahap 1 : Pencarian Fitur dan Algoritma Terbaik

In [ ]:
feature = [("IS09", X_IS09), ("IS10", X_IS10), ("IS13", X_IS13), ("eGeMAPS", X_eGeMAPS), ("emobase", X_emobase)]
label = y
model = ["lstm", "gru"]
n = 5

In [ ]:
def run_tahap_1(feature, label, model, train_index, n):
  performances = []
  for i in range(n):
    p = []
    for f in feature:
      X_train, y_train = f[1][train_index], label[train_index]
      for m in model:
        loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], m)
        p.append((f[0] + ' ' + m, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_1(feature, label, model, train_index, n)

Epoch 1/10
11/11 [==============================] - 0s 6ms/step - loss: 1.5430 - accuracy: 0.3460
Epoch 1/10
11/11 [==============================] - 0s 7ms/step - loss: 1.4841 - accuracy: 0.3724
Epoch 1/10
11/11 [==============================] - 1s 6ms/step - loss: 1.4713 - accuracy: 0.4059
Epoch 1/10
11/11 [==============================] - 1s 8ms/step - loss: 1.4372 - accuracy: 0.4147
Epoch 1/10
11/11 [==============================] - 1s 7ms/step - loss: 1.5155 - accuracy: 0.3500
Loss Score :  1.4902 Accuracy Score :  0.3778
Epoch 1/10
11/11 [==============================] - 0s 6ms/step - loss: 1.5274 - accuracy: 0.3431
Epoch 1/10
11/11 [==============================] - 0s 6ms/step - loss: 1.4783 - accuracy: 0.3695
Epoch 1/10
11/11 [==============================] - 2s 7ms/step - loss: 1.4458 - accuracy: 0.4029
Epoch 1/10
11/11 [==============================] - 2s 6ms/step - loss: 1.4368 - accuracy: 0.3500
Epoch 1/10
11/11 [==============================] - 2s 7ms/step - loss: 

KeyboardInterrupt: ignored

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
  print(res['Accuracy Score'])

In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

## Tahap 2 : Penentuan Set Fitur Terbaik

In [ ]:
feature = feature_eGeMAPS
label = abstraksi
n = 5

### Pemilihan subset n fitur secara acak

In [ ]:
def get_subset_features(features, idxs):
  new_features = []
  for feature in features:
    new_features.append(feature[idxs])
  return np.array(new_features)

In [ ]:
list_idx = np.arange(0, feature.shape[1], 1)
random.seed(42)
rndm_n = random.sample(list(list_idx), 10)
rndm_idx = [] 
for n in rndm_n:
  random.seed(42)
  l = random.sample(list(list_idx), n)
  idx = np.array(l)
  rndm_idx.append(idx)

In [ ]:
def run_tahap_2_1(feature, label, rndm_idx, train_index, n):
  performances = []
  for i in range(n):
    p = []
    for idx in rndm_idx:

      subset_features = get_subset_features(feature, idx)
      sequential_subset_features = construct_sequential(id, subset_features)
      
      X_subset = sequential_subset_features
      X_subset = tf.keras.preprocessing.sequence.pad_sequences(X_subset, padding="post", dtype='float32')

      X_train, y_train = X_subset[train_index], y[train_index]

      loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], "lstm")
      
      p.append((idx, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_2_1(feature, label, rndm_idx, train_index, n)

Epoch 1/10
137/137 [==============================] - 7s 20ms/step - loss: 1.5151 - accuracy: 0.3337 - val_loss: 1.4865 - val_accuracy: 0.3431
Epoch 2/10
11/11 [==============================] - 0s 4ms/step - loss: 1.5149 - accuracy: 0.3343
Epoch 1/10
137/137 [==============================] - 7s 21ms/step - loss: 1.8273 - accuracy: 0.2363 - val_loss: 1.5307 - val_accuracy: 0.2551
Epoch 2/10
137/137 [==============================] - 1s 10ms/step - loss: 1.4820 - accuracy: 0.3179 - val_loss: 1.4668 - val_accuracy: 0.3812
Epoch 3/10
137/137 [==============================] - 1s 10ms/step - loss: 1.4008 - accuracy: 0.3863 - val_loss: 1.4504 - val_accuracy: 0.3842
Epoch 4/10
11/11 [==============================] - 0s 4ms/step - loss: 1.4523 - accuracy: 0.3754
Epoch 1/10
137/137 [==============================] - 6s 18ms/step - loss: 1.8352 - accuracy: 0.1820 - val_loss: 1.5628 - val_accuracy: 0.3088
Epoch 2/10
137/137 [==============================] - 1s 10ms/step - loss: 1.5648 - accur

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['N Subset', 'Loss Score', 'Accuracy Score'])
  print(res)

In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Feature + Model', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

### Pengurangan Low Level Descriptor (LLD)

In [ ]:
def get_subset_features_by_lld(features, idx):
  new_features = []
  for feature in features:
    new_features.append(np.delete(feature, idx))
  return np.array(new_features)

In [ ]:
def get_deleted_idx(list_idx):
  new_list = []
  for idx in list_idx:
    new_list.extend(np.arange(idx[0], idx[1] + 1))
  return np.array(new_list)

In [ ]:
def train_new_features(list_del_idx):
  del_idx = get_deleted_idx(list_del_idx)
  subset_features = get_subset_features_by_lld(features, del_idx)
  sequential_subset_features = construct_sequential(id, subset_features)
    
  X_subset = sequential_subset_features
  X_subset = tf.keras.preprocessing.sequence.pad_sequences(X_subset, padding="post", dtype='float32')

  X_train, y_train = X_subset[train_index], y[train_index]

  loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], "lstm")
  return loss_score, accuracy_score

In [ ]:
performances = []

In [ ]:
LLD, list_del_idx = "Sum of auditory spectrum (loudness)", [(0, 30), (124, 154), (4131, 4153), (5488, 5502)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 28ms/step - loss: 2.0816 - accuracy: 0.0827 - val_loss: 1.5933 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4972 - accuracy: 0.3827 - val_loss: 1.4852 - val_accuracy: 0.3372
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4124 - accuracy: 0.3785 - val_loss: 1.4778 - val_accuracy: 0.3372
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4164 - accuracy: 0.3746 - val_loss: 1.4728 - val_accuracy: 0.3372
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4126 - accuracy: 0.3586 - val_loss: 1.4714 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4102 - accuracy: 0.3930 - val_loss: 1.4694 - val_accuracy: 0.3343
Epoch 7/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4698 - accuracy: 0.3402
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Sum of RASTA-style filtered auditory spectrum", [(31, 61), (155, 185), (4154, 4176), (5503, 5517)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 24ms/step - loss: 2.0351 - accuracy: 0.0489 - val_loss: 1.6310 - val_accuracy: 0.3196
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5359 - accuracy: 0.3320 - val_loss: 1.5130 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4416 - accuracy: 0.3671 - val_loss: 1.4860 - val_accuracy: 0.3343
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4045 - accuracy: 0.3952 - val_loss: 1.4784 - val_accuracy: 0.3343
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4214 - accuracy: 0.3922 - val_loss: 1.4755 - val_accuracy: 0.3314
Epoch 6/10
11/11 [==============================] - 0s 10ms/step - loss: 1.4769 - accuracy: 0.3343
Epoch 1/10
137/137 [==============================] - 8s 24ms/step - loss: 1.7268 - accuracy: 0.2312 - val_loss: 1.5710 - val_accuracy: 0.3109
Epoch 2/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "RMS Energy", [(62, 92), (186, 216), (4177, 4199), (5518, 5532)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 2.0151 - accuracy: 0.0775 - val_loss: 1.6306 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 15ms/step - loss: 1.5398 - accuracy: 0.3851 - val_loss: 1.5115 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4289 - accuracy: 0.3753 - val_loss: 1.4865 - val_accuracy: 0.3343
Epoch 4/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4082 - accuracy: 0.3733 - val_loss: 1.4806 - val_accuracy: 0.3343
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4102 - accuracy: 0.3805 - val_loss: 1.4771 - val_accuracy: 0.3343
Epoch 6/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4784 - accuracy: 0.3343
Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.9287 - accuracy: 0.0796 - val_loss: 1.6113 - val_accuracy: 0.3284
Epoch 2/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Zero-Crossing Rate", [(93, 123), (217, 237), (4200, 4222), (5533, 5547)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.8166 - accuracy: 0.2037 - val_loss: 1.5755 - val_accuracy: 0.2639
Epoch 2/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4874 - accuracy: 0.3127 - val_loss: 1.5084 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4312 - accuracy: 0.3820 - val_loss: 1.4909 - val_accuracy: 0.3109
Epoch 4/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4225 - accuracy: 0.3648 - val_loss: 1.4862 - val_accuracy: 0.3343
Epoch 5/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4879 - accuracy: 0.3343
Epoch 1/10
137/137 [==============================] - 8s 26ms/step - loss: 1.7342 - accuracy: 0.2816 - val_loss: 1.5856 - val_accuracy: 0.3226
Epoch 2/10
137/137 [==============================] - 2s 15ms/step - loss: 1.5214 - accuracy: 0.3386 - val_loss: 1.5022 - val_accuracy: 0.3578
Epoch 3/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "RASTA-style auditory spectrum, bands 1-26", [(248, 1053), (1953, 2758), (4223, 4820), (5548, 5937)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 23ms/step - loss: 1.5606 - accuracy: 0.3808 - val_loss: 1.5105 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4561 - accuracy: 0.3687 - val_loss: 1.4937 - val_accuracy: 0.3372
Epoch 3/10
137/137 [==============================] - 2s 12ms/step - loss: 1.4139 - accuracy: 0.3566 - val_loss: 1.4859 - val_accuracy: 0.3343
Epoch 4/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4294 - accuracy: 0.3802 - val_loss: 1.4796 - val_accuracy: 0.3372
Epoch 5/10
137/137 [==============================] - 2s 12ms/step - loss: 1.4279 - accuracy: 0.3916 - val_loss: 1.4789 - val_accuracy: 0.3372
Epoch 6/10
11/11 [==============================] - 0s 8ms/step - loss: 1.4802 - accuracy: 0.3372
Epoch 1/10
137/137 [==============================] - 7s 23ms/step - loss: 1.7960 - accuracy: 0.3821 - val_loss: 1.4756 - val_accuracy: 0.3783
Epoch 2/10
137/137 [========================

In [ ]:
LLD, list_del_idx = "Spectral energy 250–650 Hz, 1000–4000 Hz", [(248, 1053), (1953, 2758), (4223, 4820), (5548, 5937)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 23ms/step - loss: 1.8617 - accuracy: 0.1435 - val_loss: 1.6459 - val_accuracy: 0.3314
Epoch 2/10
137/137 [==============================] - 2s 13ms/step - loss: 1.5770 - accuracy: 0.3869 - val_loss: 1.5393 - val_accuracy: 0.3314
Epoch 3/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4622 - accuracy: 0.4025 - val_loss: 1.5031 - val_accuracy: 0.3314
Epoch 4/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4259 - accuracy: 0.4133 - val_loss: 1.4870 - val_accuracy: 0.3167
Epoch 5/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4185 - accuracy: 0.3445 - val_loss: 1.4843 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4105 - accuracy: 0.3929 - val_loss: 1.4798 - val_accuracy: 0.3314
Epoch 7/10
11/11 [==============================] - 0s 8ms/step - loss: 1.4799 - accuracy: 0.3284
Epoch 1/10
137/137 [========================

In [ ]:
LLD, list_del_idx = "Spectral Roll Off Point 0.25, 0.50, 0.75, 0.90", [(1116, 1239), (2821, 2944), (4867, 4958), (5968, 6027)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 28ms/step - loss: 1.6910 - accuracy: 0.2181 - val_loss: 1.4976 - val_accuracy: 0.3255
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4423 - accuracy: 0.3760 - val_loss: 1.4849 - val_accuracy: 0.3372
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4061 - accuracy: 0.3849 - val_loss: 1.4821 - val_accuracy: 0.3372
Epoch 4/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4848 - accuracy: 0.3314
Epoch 1/10
137/137 [==============================] - 8s 27ms/step - loss: 1.6389 - accuracy: 0.1821 - val_loss: 1.4975 - val_accuracy: 0.3783
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4531 - accuracy: 0.3834 - val_loss: 1.4651 - val_accuracy: 0.3812
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4213 - accuracy: 0.3737 - val_loss: 1.4577 - val_accuracy: 0.3724
Epoch 4/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Flux", [(1240, 1270), (2945, 2975), (4959, 4981), (6028, 6042)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.6938 - accuracy: 0.2112 - val_loss: 1.5644 - val_accuracy: 0.2815
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5042 - accuracy: 0.3436 - val_loss: 1.5026 - val_accuracy: 0.3255
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4376 - accuracy: 0.3811 - val_loss: 1.4863 - val_accuracy: 0.3255
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4202 - accuracy: 0.3683 - val_loss: 1.4837 - val_accuracy: 0.3255
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3961 - accuracy: 0.3864 - val_loss: 1.4822 - val_accuracy: 0.3255
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4001 - accuracy: 0.3811 - val_loss: 1.4804 - val_accuracy: 0.3255
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4349 - accuracy: 0.3618 - val_loss: 1.4795 - val_accuracy: 0.3255

In [ ]:
LLD, list_del_idx = "Spectral Centroid", [(1271, 1301), (2976, 3006), (4982, 5004), (6043, 6057)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 24ms/step - loss: 1.6407 - accuracy: 0.3738 - val_loss: 1.5258 - val_accuracy: 0.3314
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4371 - accuracy: 0.3943 - val_loss: 1.4847 - val_accuracy: 0.3284
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4303 - accuracy: 0.3526 - val_loss: 1.4825 - val_accuracy: 0.3314
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4225 - accuracy: 0.3708 - val_loss: 1.4812 - val_accuracy: 0.3255
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3939 - accuracy: 0.3706 - val_loss: 1.4793 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4155 - accuracy: 0.3884 - val_loss: 1.4783 - val_accuracy: 0.3343
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3978 - accuracy: 0.3946 - val_loss: 1.4750 - val_accuracy: 0.3343

In [ ]:
LLD, list_del_idx = "Spectral Entropy", [(1302, 1332), (3007, 3037), (5005, 5027), (6058, 6072)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 24ms/step - loss: 1.6307 - accuracy: 0.2056 - val_loss: 1.4978 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4613 - accuracy: 0.3770 - val_loss: 1.4867 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3931 - accuracy: 0.3983 - val_loss: 1.4805 - val_accuracy: 0.3343
Epoch 4/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4826 - accuracy: 0.3343
Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.9511 - accuracy: 0.0676 - val_loss: 1.6294 - val_accuracy: 0.2815
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5686 - accuracy: 0.3129 - val_loss: 1.4941 - val_accuracy: 0.3783
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4371 - accuracy: 0.3719 - val_loss: 1.4588 - val_accuracy: 0.3754
Epoch 4/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Variance", [(1333, 1363), (3038, 3068), (5028, 5050), (6073, 6087)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.7427 - accuracy: 0.2421 - val_loss: 1.5953 - val_accuracy: 0.3255
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5500 - accuracy: 0.3222 - val_loss: 1.5460 - val_accuracy: 0.3196
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4638 - accuracy: 0.3558 - val_loss: 1.5248 - val_accuracy: 0.3196
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4394 - accuracy: 0.3569 - val_loss: 1.5150 - val_accuracy: 0.3255
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4387 - accuracy: 0.3543 - val_loss: 1.5114 - val_accuracy: 0.3196
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4416 - accuracy: 0.3470 - val_loss: 1.5083 - val_accuracy: 0.3284
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4231 - accuracy: 0.3648 - val_loss: 1.5004 - val_accuracy: 0.3255

In [ ]:
LLD, list_del_idx = "Spectral Skewness", [(1364, 1394), (3069, 3099), (5051, 5073), (6088, 6102)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 28ms/step - loss: 1.7853 - accuracy: 0.1631 - val_loss: 1.5393 - val_accuracy: 0.3284
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4395 - accuracy: 0.3988 - val_loss: 1.4910 - val_accuracy: 0.3284
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4326 - accuracy: 0.3615 - val_loss: 1.4878 - val_accuracy: 0.3284
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4121 - accuracy: 0.3907 - val_loss: 1.4825 - val_accuracy: 0.3226
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3971 - accuracy: 0.4064 - val_loss: 1.4782 - val_accuracy: 0.3226
Epoch 6/10
11/11 [==============================] - 0s 11ms/step - loss: 1.4837 - accuracy: 0.3284
Epoch 1/10
137/137 [==============================] - 8s 27ms/step - loss: 1.8358 - accuracy: 0.1434 - val_loss: 1.5396 - val_accuracy: 0.3431
Epoch 2/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Kurtosis", [(1395, 1425), (3100, 3130), (5074, 5096), (6103, 6117)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.6837 - accuracy: 0.3021 - val_loss: 1.6047 - val_accuracy: 0.2991
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5443 - accuracy: 0.3846 - val_loss: 1.5376 - val_accuracy: 0.3314
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4692 - accuracy: 0.3846 - val_loss: 1.5088 - val_accuracy: 0.3284
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4469 - accuracy: 0.3744 - val_loss: 1.4984 - val_accuracy: 0.3284
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4275 - accuracy: 0.3813 - val_loss: 1.4916 - val_accuracy: 0.3284
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4036 - accuracy: 0.3717 - val_loss: 1.4900 - val_accuracy: 0.3284
Epoch 7/10
11/11 [==============================] - 0s 12ms/step - loss: 1.4904 - accuracy: 0.3284
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Slope", [(1426, 1456), (3131, 3161), (5097, 5119), (6118, 6132)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.8332 - accuracy: 0.1278 - val_loss: 1.6294 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5587 - accuracy: 0.3521 - val_loss: 1.5187 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4254 - accuracy: 0.3971 - val_loss: 1.4855 - val_accuracy: 0.3343
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4293 - accuracy: 0.3772 - val_loss: 1.4780 - val_accuracy: 0.3343
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4034 - accuracy: 0.4078 - val_loss: 1.4731 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4217 - accuracy: 0.3913 - val_loss: 1.4720 - val_accuracy: 0.3343
Epoch 7/10
11/11 [==============================] - 0s 10ms/step - loss: 1.4724 - accuracy: 0.3343
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Psychoacoustic Sharpness", [(1457, 1487), (3162, 3192), (5120, 5142), (6133, 6147)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.9968 - accuracy: 0.1828 - val_loss: 1.5849 - val_accuracy: 0.2698
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5267 - accuracy: 0.2927 - val_loss: 1.4843 - val_accuracy: 0.3021
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4249 - accuracy: 0.3437 - val_loss: 1.4762 - val_accuracy: 0.3196
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4295 - accuracy: 0.3602 - val_loss: 1.4748 - val_accuracy: 0.3255
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4124 - accuracy: 0.3507 - val_loss: 1.4727 - val_accuracy: 0.3255
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3924 - accuracy: 0.3849 - val_loss: 1.4713 - val_accuracy: 0.3196
Epoch 7/10
11/11 [==============================] - 0s 10ms/step - loss: 1.4715 - accuracy: 0.3343
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Spectral Harmonicity", [(1426, 1456), (3131, 3161), (5097, 5119), (6118, 6132)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.7962 - accuracy: 0.1899 - val_loss: 1.5518 - val_accuracy: 0.3196
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4671 - accuracy: 0.3725 - val_loss: 1.5000 - val_accuracy: 0.3196
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4339 - accuracy: 0.3697 - val_loss: 1.4924 - val_accuracy: 0.3167
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4252 - accuracy: 0.3794 - val_loss: 1.4904 - val_accuracy: 0.3284
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4141 - accuracy: 0.3833 - val_loss: 1.4888 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4211 - accuracy: 0.3811 - val_loss: 1.4857 - val_accuracy: 0.3284
Epoch 7/10
11/11 [==============================] - 0s 12ms/step - loss: 1.4893 - accuracy: 0.3343
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "MFCC 1-14", [(1519, 1952), (3224, 3657), (5166, 5487), (6163, 6372)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 23ms/step - loss: 1.6992 - accuracy: 0.1994 - val_loss: 1.5602 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4817 - accuracy: 0.3859 - val_loss: 1.5082 - val_accuracy: 0.3314
Epoch 3/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4468 - accuracy: 0.3652 - val_loss: 1.4978 - val_accuracy: 0.3314
Epoch 4/10
137/137 [==============================] - 2s 13ms/step - loss: 1.4207 - accuracy: 0.3705 - val_loss: 1.4923 - val_accuracy: 0.3314
Epoch 5/10
137/137 [==============================] - 2s 13ms/step - loss: 1.3943 - accuracy: 0.3762 - val_loss: 1.4872 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 13ms/step - loss: 1.3985 - accuracy: 0.3897 - val_loss: 1.4870 - val_accuracy: 0.3343
Epoch 7/10
137/137 [==============================] - 2s 13ms/step - loss: 1.3852 - accuracy: 0.4044 - val_loss: 1.4835 - val_accuracy: 0.3314

In [ ]:
LLD, list_del_idx = "F0", [(3658, 3696), (3892, 3930)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.7116 - accuracy: 0.1983 - val_loss: 1.5365 - val_accuracy: 0.2962
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4816 - accuracy: 0.3509 - val_loss: 1.4964 - val_accuracy: 0.2962
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4412 - accuracy: 0.3533 - val_loss: 1.4859 - val_accuracy: 0.3226
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4182 - accuracy: 0.3889 - val_loss: 1.4806 - val_accuracy: 0.2962
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4247 - accuracy: 0.3681 - val_loss: 1.4803 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4280 - accuracy: 0.3873 - val_loss: 1.4784 - val_accuracy: 0.3343
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4013 - accuracy: 0.3999 - val_loss: 1.4764 - val_accuracy: 0.3343

In [ ]:
LLD, list_del_idx = "Viterbi Smoothing", [(3607, 3735), (3931, 3969)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.6447 - accuracy: 0.2992 - val_loss: 1.5250 - val_accuracy: 0.3226
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4349 - accuracy: 0.3932 - val_loss: 1.4936 - val_accuracy: 0.3255
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4147 - accuracy: 0.3855 - val_loss: 1.4840 - val_accuracy: 0.3255
Epoch 4/10
11/11 [==============================] - 0s 10ms/step - loss: 1.4855 - accuracy: 0.3284
Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.6652 - accuracy: 0.1955 - val_loss: 1.4856 - val_accuracy: 0.3812
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4513 - accuracy: 0.3713 - val_loss: 1.4506 - val_accuracy: 0.3812
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3796 - accuracy: 0.3848 - val_loss: 1.4442 - val_accuracy: 0.3812
Epoch 4/10
11/11 [=========================

In [ ]:
LLD, list_del_idx = "Jitter Local, Delta", [(3736, 3813), (3970, 4047)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 7s 24ms/step - loss: 1.7558 - accuracy: 0.2527 - val_loss: 1.7120 - val_accuracy: 0.2405
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.6456 - accuracy: 0.3340 - val_loss: 1.6415 - val_accuracy: 0.2405
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5754 - accuracy: 0.3197 - val_loss: 1.5917 - val_accuracy: 0.2463
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5130 - accuracy: 0.3534 - val_loss: 1.5582 - val_accuracy: 0.3284
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4885 - accuracy: 0.3887 - val_loss: 1.5364 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4724 - accuracy: 0.3719 - val_loss: 1.5214 - val_accuracy: 0.3343
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4322 - accuracy: 0.3758 - val_loss: 1.5099 - val_accuracy: 0.3343

In [ ]:
LLD, list_del_idx = "Shimmer Local", [(3184, 3852), (4048, 4086)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.6625 - accuracy: 0.1874 - val_loss: 1.5264 - val_accuracy: 0.3314
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4815 - accuracy: 0.3761 - val_loss: 1.4835 - val_accuracy: 0.3284
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4245 - accuracy: 0.3785 - val_loss: 1.4759 - val_accuracy: 0.3284
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4256 - accuracy: 0.3904 - val_loss: 1.4756 - val_accuracy: 0.3314
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3935 - accuracy: 0.4010 - val_loss: 1.4727 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 15ms/step - loss: 1.4070 - accuracy: 0.3823 - val_loss: 1.4717 - val_accuracy: 0.3314
Epoch 7/10
11/11 [==============================] - 0s 10ms/step - loss: 1.4716 - accuracy: 0.3314
Epoch 1/10
137/137 [=======================

In [ ]:
LLD, list_del_idx = "Probability of voicing logarithmic HNR", [(3853, 3891), (4087, 4125)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.6768 - accuracy: 0.3140 - val_loss: 1.5669 - val_accuracy: 0.3343
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4827 - accuracy: 0.3764 - val_loss: 1.5080 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4564 - accuracy: 0.3820 - val_loss: 1.4865 - val_accuracy: 0.3314
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4056 - accuracy: 0.3884 - val_loss: 1.4799 - val_accuracy: 0.3314
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4044 - accuracy: 0.3837 - val_loss: 1.4781 - val_accuracy: 0.3314
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3919 - accuracy: 0.3845 - val_loss: 1.4774 - val_accuracy: 0.3314
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3864 - accuracy: 0.3857 - val_loss: 1.4752 - val_accuracy: 0.3314

In [ ]:
LLD, list_del_idx = "F0 envelope", [(4126, 4130)]
loss_score, accuracy_score = train_new_features(list_del_idx)
performances.append((LLD, loss_score, accuracy_score))

Epoch 1/10
137/137 [==============================] - 8s 25ms/step - loss: 1.7086 - accuracy: 0.2890 - val_loss: 1.5885 - val_accuracy: 0.3402
Epoch 2/10
137/137 [==============================] - 2s 14ms/step - loss: 1.5184 - accuracy: 0.3624 - val_loss: 1.5196 - val_accuracy: 0.3372
Epoch 3/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4457 - accuracy: 0.3566 - val_loss: 1.4980 - val_accuracy: 0.3402
Epoch 4/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4561 - accuracy: 0.3540 - val_loss: 1.4885 - val_accuracy: 0.3372
Epoch 5/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4422 - accuracy: 0.3510 - val_loss: 1.4812 - val_accuracy: 0.3402
Epoch 6/10
137/137 [==============================] - 2s 14ms/step - loss: 1.4321 - accuracy: 0.3648 - val_loss: 1.4771 - val_accuracy: 0.3372
Epoch 7/10
137/137 [==============================] - 2s 14ms/step - loss: 1.3779 - accuracy: 0.3982 - val_loss: 1.4732 - val_accuracy: 0.3372

In [ ]:
res = pd.DataFrame(performances, columns=['Deleted LLD', 'Loss Score', 'Accuracy Score'])
res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True)

,Deleted LLD,Loss Score,Accuracy Score
0,"Jitter Local, Delta",1.4285,0.3766
1,F0,1.4256,0.3761
2,Sum of auditory spectrum (loudness),1.4270,0.3755
3,Spectral Centroid,1.4288,0.3749
4,Spectral Kurtosis,1.4275,0.3749
5,Spectral Slope,1.4306,0.3737
6,Spectral Entropy,1.4313,0.3737
7,"Spectral energy 250–650 Hz, 1000–4000 Hz",1.4295,0.3731
8,Zero-Crossing Rate,1.4335,0.3731
9,Sum of RASTA-style filtered auditory spectrum,1.4303,0.3725


## Tahap 3 : Pencarian Arsitektur Model Terbaik

In [ ]:
model = [1, 2, 3]
X = X_eGeMAPS
label = abstraksi
n = 5

In [ ]:
def run_tahap_3(X, label, model, train_index, n):
  performances = []
  for i in range(n):
    p = []
    X_train, y_train = X[train_index], label[train_index]
    for m in model:
      loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], m)
      p.append((m, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
performances = run_tahap_3(X, label, model, train_index, n)

Epoch 1/10
137/137 [==============================] - 6s 17ms/step - loss: 1.7800 - accuracy: 0.2660 - val_loss: 1.5236 - val_accuracy: 0.3255
Epoch 2/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4474 - accuracy: 0.3559 - val_loss: 1.4801 - val_accuracy: 0.3343
Epoch 3/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4243 - accuracy: 0.3784 - val_loss: 1.4708 - val_accuracy: 0.3343
Epoch 4/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4179 - accuracy: 0.3804 - val_loss: 1.4704 - val_accuracy: 0.3343
Epoch 5/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4140 - accuracy: 0.3929 - val_loss: 1.4689 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 1s 7ms/step - loss: 1.3923 - accuracy: 0.3922 - val_loss: 1.4685 - val_accuracy: 0.3314
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 1.4685 - accuracy: 0.3314
Epoch 1/10
137/137 [=============================

In [ ]:
for p in performances:
  print("Training ke- : ", p[0])
  res = pd.DataFrame(p[1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
  print(res['Accuracy Score'])

Training ke- :  0
0    0.3737
1    0.3696
2    0.3749
Name: Accuracy Score, dtype: float64
Training ke- :  1
0    0.3743
1    0.3725
2    0.3643
Name: Accuracy Score, dtype: float64
Training ke- :  2
0    0.3684
1    0.3702
2    0.3566
Name: Accuracy Score, dtype: float64
Training ke- :  3
0    0.3731
1    0.3796
2    0.3743
Name: Accuracy Score, dtype: float64
Training ke- :  4
0    0.3749
1    0.3743
2    0.3566
Name: Accuracy Score, dtype: float64


In [ ]:
res = pd.DataFrame(performances[0][1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
for performance in performances[1:]:
  temp = pd.DataFrame(performance[1], columns=['Architecture', 'Loss Score', 'Accuracy Score'])
  res['Loss Score'] += temp['Loss Score']
  res['Accuracy Score'] += temp['Accuracy Score']
res['Loss Score'] /= n
res['Accuracy Score'] /= n
print(res.sort_values(by='Accuracy Score', ascending=False, ignore_index=True))

   Architecture  Loss Score  Accuracy Score
0             2     1.43086         0.37324
1             1     1.42706         0.37288
2             3     1.42490         0.36534


## Tahap 4 : Penentuan Parameter Model

In [ ]:
X = X_eGeMAPS
label = abstraksi

In [ ]:
n = 5

In [ ]:
def run_tahap_4(X, label, train_index, n):
  performances = []
  for i in range(n):
    p = []
    X_train, y_train = X[train_index], label[train_index]
    loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 2)
    p.append((batch_size, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

### Batch Size

In [ ]:
batch_size = 10
run_tahap_4(X, label, train_index, n)

Epoch 1/10
137/137 [==============================] - 6s 15ms/step - loss: 1.7564 - accuracy: 0.1898 - val_loss: 1.5631 - val_accuracy: 0.3314
Epoch 2/10
137/137 [==============================] - 1s 7ms/step - loss: 1.4748 - accuracy: 0.3744 - val_loss: 1.5199 - val_accuracy: 0.3255
Epoch 3/10
137/137 [==============================] - 1s 7ms/step - loss: 1.4470 - accuracy: 0.3881 - val_loss: 1.4947 - val_accuracy: 0.3284
Epoch 4/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4368 - accuracy: 0.3634 - val_loss: 1.4933 - val_accuracy: 0.3314
Epoch 5/10
137/137 [==============================] - 1s 7ms/step - loss: 1.4696 - accuracy: 0.3544 - val_loss: 1.4845 - val_accuracy: 0.3343
Epoch 6/10
137/137 [==============================] - 1s 6ms/step - loss: 1.4190 - accuracy: 0.3734 - val_loss: 1.4826 - val_accuracy: 0.3284
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 1.4826 - accuracy: 0.3284
Epoch 1/10
137/137 [=============================

[(0, [(10, 1.4227, 0.3713)]),
 (1, [(10, 1.4243, 0.3708)]),
 (2, [(10, 1.4301, 0.3719)]),
 (3, [(10, 1.4284, 0.3731)]),
 (4, [(10, 1.4366, 0.3526)])]

In [ ]:
batch_size = 32
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 5s 33ms/step - loss: 1.6419 - accuracy: 0.3515 - val_loss: 1.5870 - val_accuracy: 0.3314
Epoch 2/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5487 - accuracy: 0.3778 - val_loss: 1.5439 - val_accuracy: 0.3314
Epoch 3/10
43/43 [==============================] - 0s 8ms/step - loss: 1.5006 - accuracy: 0.3464 - val_loss: 1.5193 - val_accuracy: 0.3314
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4953 - accuracy: 0.3548 - val_loss: 1.5050 - val_accuracy: 0.3314
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4680 - accuracy: 0.3764 - val_loss: 1.4961 - val_accuracy: 0.3314
Epoch 6/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4368 - accuracy: 0.3757 - val_loss: 1.4892 - val_accuracy: 0.3314
Epoch 7/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4734 - accuracy: 0.3414 - val_loss: 1.4876 - val_accuracy: 0.3314
Epoch 8/10
43/43 [=

In [ ]:
batch_size = 64
run_tahap_4(X, label, train_index, n)

Epoch 1/10
22/22 [==============================] - 6s 63ms/step - loss: 1.6688 - accuracy: 0.3215 - val_loss: 1.5629 - val_accuracy: 0.3284
Epoch 2/10
22/22 [==============================] - 0s 8ms/step - loss: 1.5713 - accuracy: 0.3479 - val_loss: 1.5289 - val_accuracy: 0.3284
Epoch 3/10
22/22 [==============================] - 0s 8ms/step - loss: 1.5163 - accuracy: 0.3495 - val_loss: 1.5132 - val_accuracy: 0.3284
Epoch 4/10
22/22 [==============================] - 0s 7ms/step - loss: 1.4529 - accuracy: 0.4011 - val_loss: 1.5057 - val_accuracy: 0.3314
Epoch 5/10
22/22 [==============================] - 0s 8ms/step - loss: 1.5045 - accuracy: 0.3308 - val_loss: 1.4990 - val_accuracy: 0.3284
Epoch 6/10
22/22 [==============================] - 0s 8ms/step - loss: 1.4614 - accuracy: 0.3824 - val_loss: 1.4938 - val_accuracy: 0.3314
Epoch 7/10
22/22 [==============================] - 0s 7ms/step - loss: 1.4708 - accuracy: 0.3479 - val_loss: 1.4862 - val_accuracy: 0.3255
Epoch 8/10
11/11 [=

In [ ]:
batch_size = 128
performances.append(run_tahap_4(X, label, train_index, n))

Epoch 1/10
11/11 [==============================] - 6s 124ms/step - loss: 1.7379 - accuracy: 0.2125 - val_loss: 1.6498 - val_accuracy: 0.2581
Epoch 2/10
11/11 [==============================] - 0s 9ms/step - loss: 1.6556 - accuracy: 0.2410 - val_loss: 1.6157 - val_accuracy: 0.2727
Epoch 3/10
11/11 [==============================] - 0s 10ms/step - loss: 1.6119 - accuracy: 0.2859 - val_loss: 1.5931 - val_accuracy: 0.2786
Epoch 4/10
11/11 [==============================] - 0s 9ms/step - loss: 1.5941 - accuracy: 0.2745 - val_loss: 1.5757 - val_accuracy: 0.2786
Epoch 5/10
11/11 [==============================] - 0s 10ms/step - loss: 1.5639 - accuracy: 0.3270 - val_loss: 1.5578 - val_accuracy: 0.2786
Epoch 6/10
11/11 [==============================] - 0s 9ms/step - loss: 1.5326 - accuracy: 0.3076 - val_loss: 1.5471 - val_accuracy: 0.2757
Epoch 7/10
11/11 [==============================] - 0s 9ms/step - loss: 1.5175 - accuracy: 0.3067 - val_loss: 1.5355 - val_accuracy: 0.2698
Epoch 8/10
11/11

### Optimizer

In [ ]:
batch_size = 32

In [ ]:
optimizer = 'adam'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 34ms/step - loss: 2.1315 - accuracy: 0.0794 - val_loss: 1.6963 - val_accuracy: 0.2815
Epoch 2/10
43/43 [==============================] - 0s 7ms/step - loss: 1.6813 - accuracy: 0.2923 - val_loss: 1.6177 - val_accuracy: 0.2845
Epoch 3/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5965 - accuracy: 0.2999 - val_loss: 1.5432 - val_accuracy: 0.2845
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5009 - accuracy: 0.3387 - val_loss: 1.5247 - val_accuracy: 0.3372
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4685 - accuracy: 0.3651 - val_loss: 1.5209 - val_accuracy: 0.3343
Epoch 6/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4626 - accuracy: 0.3713 - val_loss: 1.4960 - val_accuracy: 0.2874
Epoch 7/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4678 - accuracy: 0.3339 - val_loss: 1.4834 - val_accuracy: 0.3431
Epoch 8/10
11/11 [=

[(0, [(32, 1.4358, 0.3755)]),
 (1, [(32, 1.4418, 0.3755)]),
 (2, [(32, 1.4573, 0.3666)]),
 (3, [(32, 1.4406, 0.3719)]),
 (4, [(32, 1.4354, 0.3737)])]

In [ ]:
optimizer = 'rmsprop'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 34ms/step - loss: 1.9098 - accuracy: 0.3403 - val_loss: 1.7392 - val_accuracy: 0.3402
Epoch 2/10
43/43 [==============================] - 0s 7ms/step - loss: 1.7164 - accuracy: 0.3572 - val_loss: 1.6266 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5715 - accuracy: 0.3678 - val_loss: 1.5659 - val_accuracy: 0.3460
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5095 - accuracy: 0.3890 - val_loss: 1.5280 - val_accuracy: 0.3314
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4803 - accuracy: 0.3589 - val_loss: 1.5112 - val_accuracy: 0.3343
Epoch 6/10
43/43 [==============================] - 0s 8ms/step - loss: 1.4926 - accuracy: 0.3446 - val_loss: 1.4943 - val_accuracy: 0.3431
Epoch 7/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4750 - accuracy: 0.3504 - val_loss: 1.4937 - val_accuracy: 0.3460
Epoch 8/10
43/43 [=

[(0, [(32, 1.4379, 0.3737)]),
 (1, [(32, 1.4334, 0.3743)]),
 (2, [(32, 1.4427, 0.3743)]),
 (3, [(32, 1.4622, 0.3713)]),
 (4, [(32, 1.4289, 0.3713)])]

In [ ]:
optimizer = 'sgd'
run_tahap_4(X, label, train_index, n)

Epoch 1/10
43/43 [==============================] - 6s 34ms/step - loss: 1.9349 - accuracy: 0.1375 - val_loss: 1.6647 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 0s 6ms/step - loss: 1.6355 - accuracy: 0.3364 - val_loss: 1.5754 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5362 - accuracy: 0.3651 - val_loss: 1.5320 - val_accuracy: 0.3314
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5113 - accuracy: 0.3481 - val_loss: 1.5118 - val_accuracy: 0.3343
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.5216 - accuracy: 0.3509 - val_loss: 1.5005 - val_accuracy: 0.3343
Epoch 6/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4808 - accuracy: 0.3610 - val_loss: 1.4943 - val_accuracy: 0.3372
Epoch 7/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4674 - accuracy: 0.3673 - val_loss: 1.4912 - val_accuracy: 0.3372
Epoch 8/10
43/43 [=

[(0, [(32, 1.4356, 0.3713)]),
 (1, [(32, 1.4517, 0.3743)]),
 (2, [(32, 1.4442, 0.3737)]),
 (3, [(32, 1.4465, 0.3743)]),
 (4, [(32, 1.4529, 0.3713)])]

## Tahap 5 : Penentuan Penggunaan Teknik Resampling

In [26]:
X = X_eGeMAPS
y = abstraksi

In [39]:
n = 5
def run_tahap_5_cv(X_train, y_train, n):
  performances = []
  for i in range(n):
    p = []
    loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 2)
    p.append((batch_size, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [ ]:
n = 5
encoder = encoder_cakapan
def run_tahap_5_randomsampling(X_train, y_train, X_test, y_test, n, encoder):
  model = None
  best_acc = 0
  for i in range(n):
    model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y.shape[1], 2, encoder)
    if best_acc < accuracy_score:
      best_acc = accuracy_score
      best_model = model
  return best_model

In [30]:
batch_size = 32
optimizer = 'adam'

### Tanpa Resampling

In [42]:
run_tahap_5_cv(X[train_index], y[train_index], n)

Epoch 1/10
43/43 [==============================] - 5s 34ms/step - loss: 1.7673 - accuracy: 0.2718 - val_loss: 1.5281 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4985 - accuracy: 0.3396 - val_loss: 1.5020 - val_accuracy: 0.3343
Epoch 3/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4842 - accuracy: 0.3492 - val_loss: 1.4928 - val_accuracy: 0.3343
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4648 - accuracy: 0.3517 - val_loss: 1.4836 - val_accuracy: 0.3343
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4803 - accuracy: 0.3241 - val_loss: 1.4928 - val_accuracy: 0.3343
Epoch 6/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4511 - accuracy: 0.3915 - val_loss: 1.4888 - val_accuracy: 0.3343
Epoch 7/10
11/11 [==============================] - 0s 3ms/step - loss: 1.4836 - accuracy: 0.3343
Epoch 1/10
43/43 [==============================] - 5s 35ms/s

[(0, [(32, 1.4244, 0.3725)]),
 (1, [(32, 1.4243, 0.3743)]),
 (2, [(32, 1.4222, 0.3737)]),
 (3, [(32, 1.4256, 0.3737)]),
 (4, [(32, 1.4245, 0.3737)])]

In [ ]:
best_model = None
best_acc = 0
for i in range(n):
  model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y.shape[1], 3, encoder)
  if best_acc < accuracy_score:
    best_acc = accuracy_score
    best_model = model
best_model.evaluate(X_test, y_test)

In [ ]:
best_model.save(drive_dir + "/Eksperimen/Model/model_akustik_langsung_6.h5")
best_model.evaluate(X_test, y_test)

10/10 [==============================] - 0s 9ms/step - loss: 1.6329 - accuracy: 0.3754


[1.6329095363616943, 0.37541529536247253]

In [ ]:
best_model = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_langsung_6.h5")
pred = best_model.predict(X_test)
y_pred = encoder.inverse_transform(pred)
y_true = encoder.inverse_transform(y_test)
print(classification_report(y_true, y_pred, digits=4))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.5000    0.0238    0.0455        42
       Sedih     0.3614    0.3488    0.3550        86
      Senang     0.4176    0.6726    0.5153       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.2609    0.1200    0.1644        50

    accuracy                         0.3754       301
   macro avg     0.2566    0.1942    0.1800       301
weighted avg     0.3731    0.3754    0.3285       301



### Dengan Resampling

In [34]:
X_train = X[train_index]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]))

In [36]:
oversample = SMOTE(kind='regular', k_neighbors=2)
X_train_resampling, y_train_resampling = oversample.fit_resample(X_train, y[train_index])
X_train_resampling = X_train_resampling.reshape(3828, 15, 88)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [41]:
run_tahap_5_cv(X_train_resampling, y_train_resampling, n)

Epoch 1/10
96/96 [==============================] - 40s 19ms/step - loss: 1.8943 - accuracy: 0.1821 - val_loss: 1.7453 - val_accuracy: 0.2441
Epoch 2/10
96/96 [==============================] - 1s 7ms/step - loss: 1.7662 - accuracy: 0.2329 - val_loss: 1.6905 - val_accuracy: 0.3251
Epoch 3/10
96/96 [==============================] - 1s 8ms/step - loss: 1.7307 - accuracy: 0.2530 - val_loss: 1.6458 - val_accuracy: 0.3094
Epoch 4/10
96/96 [==============================] - 1s 7ms/step - loss: 1.6676 - accuracy: 0.3113 - val_loss: 1.6309 - val_accuracy: 0.3159
Epoch 5/10
96/96 [==============================] - 1s 7ms/step - loss: 1.6371 - accuracy: 0.3103 - val_loss: 1.5537 - val_accuracy: 0.3368
Epoch 6/10
96/96 [==============================] - 1s 7ms/step - loss: 1.6331 - accuracy: 0.2915 - val_loss: 1.5526 - val_accuracy: 0.3433
Epoch 7/10
96/96 [==============================] - 1s 7ms/step - loss: 1.5672 - accuracy: 0.3009 - val_loss: 1.5636 - val_accuracy: 0.3303
Epoch 8/10
96/96 [

[(0, [(32, 1.4888, 0.3333)]),
 (1, [(32, 1.4984, 0.3467)]),
 (2, [(32, 1.4957, 0.3401)]),
 (3, [(32, 1.5139, 0.3276)]),
 (4, [(32, 1.4907, 0.3372)])]

## Tahap 6 : Penerapan Normalisasi

In [46]:
scaler = MinMaxScaler()
feature_eGeMAPS = scaler.fit_transform(feature_eGeMAPS)
sequential_features_eGeMAPS = construct_sequential(id, feature_eGeMAPS)
X_eGeMAPS = tf.keras.preprocessing.sequence.pad_sequences(sequential_features_eGeMAPS, padding="post", dtype='float32')
X = X_eGeMAPS
X_eGeMAPS.shape

(2003, 15, 88)

In [44]:
batch_size = 32
optimizer = 'adam'

In [45]:
n = 5
def run_tahap_6_cv(X_train, y_train, n):
  performances = []
  for i in range(n):
    p = []
    loss_score, accuracy_score = train_5fold_cv(X_train, y_train, X_train.shape[1:], y.shape[1], 2)
    p.append((batch_size, loss_score, accuracy_score))
    performances.append((i, p))
  return performances

In [58]:
run_tahap_6_cv(X[train_index], y[train_index], 5)

Epoch 1/10
43/43 [==============================] - 5s 44ms/step - loss: 1.5341 - accuracy: 0.3451 - val_loss: 1.4645 - val_accuracy: 0.3343
Epoch 2/10
43/43 [==============================] - 0s 8ms/step - loss: 1.3968 - accuracy: 0.3817 - val_loss: 1.4694 - val_accuracy: 0.3636
Epoch 3/10
43/43 [==============================] - 0s 7ms/step - loss: 1.4148 - accuracy: 0.3737 - val_loss: 1.4317 - val_accuracy: 0.3578
Epoch 4/10
43/43 [==============================] - 0s 7ms/step - loss: 1.3902 - accuracy: 0.3765 - val_loss: 1.4352 - val_accuracy: 0.3460
Epoch 5/10
43/43 [==============================] - 0s 7ms/step - loss: 1.3843 - accuracy: 0.3844 - val_loss: 1.4256 - val_accuracy: 0.3754
Epoch 6/10
43/43 [==============================] - 0s 8ms/step - loss: 1.4066 - accuracy: 0.3541 - val_loss: 1.4215 - val_accuracy: 0.3490
Epoch 7/10
43/43 [==============================] - 0s 7ms/step - loss: 1.3674 - accuracy: 0.3932 - val_loss: 1.4066 - val_accuracy: 0.4047
Epoch 8/10
43/43 [=

[(0, [(32, 1.3727, 0.379)]),
 (1, [(32, 1.3807, 0.3884)]),
 (2, [(32, 1.3829, 0.3848)]),
 (3, [(32, 1.3807, 0.3872)]),
 (4, [(32, 1.3783, 0.3772)])]

## Tahap 7 : Evaluasi Akhir Model Terbaik

In [48]:
n = 5
encoder = encoder_cakapan
def evaluate_model(X_train, y_train, X_test, y_test, n, encoder):
  best_model = None
  best_acc = 0
  for i in range(n):
    model, accuracy_score = train_stratified_random_sampling(X_train, y_train, X_test, y_test, X_train.shape[1:], y.shape[1], 2, encoder)
    if best_acc < accuracy_score:
      best_acc = accuracy_score
      best_model = model
  return best_model

### 6 Kelas Emosi

In [ ]:
X = X_eGeMAPS
y = abstraksi
print(X.shape, y.shape)

(2003, 15, 88) (2003, 6)


In [ ]:
best_model = evaluate_model(X[train_index], y[train_index], X[test_index], y[test_index], 1, encoder)

Epoch 1/10
171/171 [==============================] - 5s 13ms/step - loss: 1.4858 - accuracy: 0.3622 - val_loss: 1.4003 - val_accuracy: 0.3754
Epoch 2/10
171/171 [==============================] - 1s 7ms/step - loss: 1.4066 - accuracy: 0.3729 - val_loss: 1.3852 - val_accuracy: 0.3821
Epoch 3/10
171/171 [==============================] - 1s 6ms/step - loss: 1.3933 - accuracy: 0.3946 - val_loss: 1.3782 - val_accuracy: 0.4186
Epoch 4/10
171/171 [==============================] - 1s 6ms/step - loss: 1.4123 - accuracy: 0.3735 - val_loss: 1.3707 - val_accuracy: 0.4120
Epoch 5/10
171/171 [==============================] - 1s 6ms/step - loss: 1.3821 - accuracy: 0.3669 - val_loss: 1.3792 - val_accuracy: 0.3787
Epoch 6/10
171/171 [==============================] - 1s 7ms/step - loss: 1.3640 - accuracy: 0.3847 - val_loss: 1.3624 - val_accuracy: 0.4053
Epoch 7/10
171/171 [==============================] - 1s 7ms/step - loss: 1.3802 - accuracy: 0.3889 - val_loss: 1.3692 - val_accuracy: 0.4120
Epoch

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
pred = best_model.predict(X[test_index])
y_pred = encoder.inverse_transform(pred)
y_true = encoder.inverse_transform(y[test_index])
print(classification_report(y_true, y_pred, digits=4))

Epoch 1/10
171/171 [==============================] - 6s 14ms/step - loss: 1.5043 - accuracy: 0.3232 - val_loss: 1.4392 - val_accuracy: 0.3223
Epoch 2/10
171/171 [==============================] - 1s 6ms/step - loss: 1.4115 - accuracy: 0.3819 - val_loss: 1.4339 - val_accuracy: 0.2924
Epoch 3/10
171/171 [==============================] - 1s 7ms/step - loss: 1.4094 - accuracy: 0.3543 - val_loss: 1.3806 - val_accuracy: 0.4086
Epoch 4/10
171/171 [==============================] - 1s 6ms/step - loss: 1.3888 - accuracy: 0.3860 - val_loss: 1.4453 - val_accuracy: 0.3754
Epoch 5/10
171/171 [==============================] - 1s 6ms/step - loss: 1.3752 - accuracy: 0.3571 - val_loss: 1.3655 - val_accuracy: 0.3920
Epoch 6/10
171/171 [==============================] - 1s 6ms/step - loss: 1.3684 - accuracy: 0.3700 - val_loss: 1.3999 - val_accuracy: 0.3522
Epoch 7/10
171/171 [==============================] - 1s 7ms/step - loss: 1.3831 - accuracy: 0.3724 - val_loss: 1.4162 - val_accuracy: 0.3455
Epoch

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
best_model.save(drive_dir + "/Eksperimen/Model/model_akustik_langsung_6.h5")

In [ ]:
best_model2 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_langsung_6.h5")
pred = best_model2.predict(X[test_index])
y_pred = encoder.inverse_transform(pred)
y_true = encoder.inverse_transform(y[test_index])
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       Jijik     0.0000    0.0000    0.0000         1
       Marah     0.3333    0.4286    0.3750        42
       Sedih     0.4176    0.4419    0.4294        86
      Senang     0.4286    0.5310    0.4743       113
       Takut     0.0000    0.0000    0.0000         9
    Terkejut     0.6875    0.2200    0.3333        50

    accuracy                         0.4219       301
   macro avg     0.3112    0.2702    0.2687       301
weighted avg     0.4409    0.4219    0.4084       301

[[ 0  0  0  1  0  0]
 [ 0 18  9 14  0  1]
 [ 0  9 38 39  0  0]
 [ 0 15 34 60  0  4]
 [ 0  1  3  5  0  0]
 [ 0 11  7 21  0 11]]


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 4 Kelas Emosi

In [ ]:
def get_label_idxs(list, label):
  idxs = []
  for i in range(len(list)):
    for l in label:
      if list[i] == l:
        idxs.append(i)
  return np.array(idxs)

In [ ]:
dec_abstraksi = encoder_cakapan.inverse_transform(abstraksi)
del_idxs = get_label_idxs(dec_abstraksi, ["Jijik", "Takut"])
updated_features = np.delete(sequential_features_eGeMAPS, del_idxs)
updated_labels = np.delete(dec_abstraksi, del_idxs)
updated_labels = updated_labels.reshape(-1, 1)
print(updated_features.shape, updated_labels.shape)

(1936,) (1936, 1)


In [ ]:
new_encoder_cakapan = OneHotEncoder(sparse=False)
updated_labels = updated_labels.reshape(-1, 1)
updated_labels = new_encoder_cakapan.fit_transform(updated_labels)
updated_labels.shape

(1936, 4)

In [ ]:
X = tf.keras.preprocessing.sequence.pad_sequences(updated_features, padding="post", dtype='float32')
y = tf.keras.preprocessing.sequence.pad_sequences(updated_labels, padding="post", dtype='float32')
print(X.shape, y.shape)

(1936, 15, 88) (1936, 4)


In [ ]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
for train_index, test_index in sss.split(X, y):
  train_index, test_index = train_index, test_index
print(X.shape, y.shape)

(1936, 15, 88) (1936, 4)


In [ ]:
models = []
for i in range(20):
  print("Model ke-", i)
  best_model = evaluate_model(X[train_index], y[train_index], X[test_index], y[test_index], 1, new_encoder_cakapan)
  models.append(best_model)

Model ke- 0
Epoch 1/10
165/165 [==============================] - 6s 13ms/step - loss: 1.3371 - accuracy: 0.3824 - val_loss: 1.2913 - val_accuracy: 0.3883
Epoch 2/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2693 - accuracy: 0.3980 - val_loss: 1.2637 - val_accuracy: 0.3883
Epoch 3/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2671 - accuracy: 0.4072 - val_loss: 1.2494 - val_accuracy: 0.3952
Epoch 4/10
165/165 [==============================] - 1s 7ms/step - loss: 1.2683 - accuracy: 0.3862 - val_loss: 1.3209 - val_accuracy: 0.3711
Epoch 5/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2809 - accuracy: 0.4141 - val_loss: 1.2503 - val_accuracy: 0.4158
Epoch 6/10
165/165 [==============================] - 1s 7ms/step - loss: 1.2671 - accuracy: 0.4047 - val_loss: 1.2337 - val_accuracy: 0.4089
Epoch 7/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2271 - accuracy: 0.4145 - val_loss: 1.2397 - val_accuracy: 

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
165/165 [==============================] - 6s 14ms/step - loss: 1.3477 - accuracy: 0.3598 - val_loss: 1.2965 - val_accuracy: 0.3883
Epoch 2/10
165/165 [==============================] - 1s 7ms/step - loss: 1.2907 - accuracy: 0.3972 - val_loss: 1.2904 - val_accuracy: 0.3643
Epoch 3/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2808 - accuracy: 0.3787 - val_loss: 1.2552 - val_accuracy: 0.3986
Epoch 4/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2611 - accuracy: 0.3800 - val_loss: 1.2599 - val_accuracy: 0.3883
Epoch 5/10
165/165 [==============================] - 1s 7ms/step - loss: 1.2712 - accuracy: 0.4004 - val_loss: 1.2380 - val_accuracy: 0.4158
Epoch 6/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2599 - accuracy: 0.3940 - val_loss: 1.2725 - val_accuracy: 0.3849
Epoch 7/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2576 - accuracy: 0.4104 - val_loss: 1.2584 - val_accuracy: 0.3986
Epoch

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/10
165/165 [==============================] - 6s 13ms/step - loss: 1.3531 - accuracy: 0.3570 - val_loss: 1.2917 - val_accuracy: 0.3368
Epoch 2/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2873 - accuracy: 0.3865 - val_loss: 1.2860 - val_accuracy: 0.3643
Epoch 3/10
165/165 [==============================] - 1s 7ms/step - loss: 1.2822 - accuracy: 0.3992 - val_loss: 1.2599 - val_accuracy: 0.3883
Epoch 4/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2814 - accuracy: 0.3612 - val_loss: 1.2454 - val_accuracy: 0.4021
Epoch 5/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2687 - accuracy: 0.4118 - val_loss: 1.2439 - val_accuracy: 0.4021
Epoch 6/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2696 - accuracy: 0.3846 - val_loss: 1.2234 - val_accuracy: 0.4364
Epoch 7/10
165/165 [==============================] - 1s 6ms/step - loss: 1.2351 - accuracy: 0.4044 - val_loss: 1.2552 - val_accuracy: 0.4021
Epoch

In [ ]:
pred = models[6].predict(X[test_index])
y_pred = new_encoder_cakapan.inverse_transform(pred)
y_true = new_encoder_cakapan.inverse_transform(y[test_index])
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       Marah     0.3462    0.4286    0.3830        42
       Sedih     0.3983    0.5465    0.4608        86
      Senang     0.4494    0.3540    0.3960       113
    Terkejut     0.5312    0.3400    0.4146        50

    accuracy                         0.4192       291
   macro avg     0.4313    0.4173    0.4136       291
weighted avg     0.4335    0.4192    0.4165       291

[[18 12 10  2]
 [ 8 47 29  2]
 [15 47 40 11]
 [11 12 10 17]]


In [ ]:
models[6].save(drive_dir + "/Eksperimen/Model/model_akustik_langsung_4.h5")

In [ ]:
best_model2 = keras.models.load_model(drive_dir + "/Eksperimen/Model/model_akustik_langsung_4.h5")
pred = best_model2.predict(X[test_index])
y_pred = new_encoder_cakapan.inverse_transform(pred)
y_true = new_encoder_cakapan.inverse_transform(y[test_index])
print(classification_report(y_true, y_pred, digits=4))
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

       Marah     0.3462    0.4286    0.3830        42
       Sedih     0.3983    0.5465    0.4608        86
      Senang     0.4494    0.3540    0.3960       113
    Terkejut     0.5312    0.3400    0.4146        50

    accuracy                         0.4192       291
   macro avg     0.4313    0.4173    0.4136       291
weighted avg     0.4335    0.4192    0.4165       291

[[18 12 10  2]
 [ 8 47 29  2]
 [15 47 40 11]
 [11 12 10 17]]
